In [4]:
l = (lambda x:x+2, 3)
l[0](5)

7

In [ ]:
class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.val = 0
        self.addLazy = 0
        self.mulLazy = 1


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)
        self.Mod = 10**9 + 7

    def pushDown(self, node: Node):
        if not node.left:
            node.left = Node(node.si, node.mi)
        if not node.right:
            node.right = Node(node.mi + 1, node.ei)
        if node.addLazy == 0 and node.mulLazy == 1:
            return
        node.left.val = node.left.val * node.mulLazy + (node.mi - node.si + 1) * node.addLazy
        node.right.val = node.right.val * node.mulLazy + (node.ei - node.mi) * node.addLazy
        node.left.mulLazy *= node.mulLazy
        node.right.mulLazy *= node.mulLazy
        node.left.addLazy = node.left.addLazy * node.mulLazy + node.addLazy
        node.right.addLazy = node.right.addLazy * node.mulLazy + node.addLazy
        node.left.val %= self.Mod
        node.right.val %= self.Mod
        node.left.mulLazy %= self.Mod
        node.right.mulLazy %= self.Mod
        node.left.addLazy %= self.Mod
        node.right.addLazy %= self.Mod
        node.addLazy = 0
        node.mulLazy = 1

    def update(self, node: Node, l, r, val, op):  # op:0为+ 1为乘
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            if not op:
                node.val += (node.ei - node.si + 1) * val
                node.addLazy += val
                node.val %= self.Mod
                node.addLazy %= self.Mod
            else:
                node.val *= val
                node.addLazy *= val
                node.mulLazy *= val
                node.val %= self.Mod
                node.addLazy %= self.Mod
                node.mulLazy %= self.Mod
            return
        self.pushDown(node)
        if node.mi >= l:
            self.update(node.left, l, r, val, op)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val, op)
        node.val = node.left.val + node.right.val
        node.val %= self.Mod

    def query(self, node: Node, l, r):
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans % self.Mod


class Fancy:

    def __init__(self):
        self.st = SegmentTree(0, int(1e5))
        self.cnt = 0

    def append(self, val: int) -> None:
        self.st.update(self.st.root, self.cnt, self.cnt, val, 0)
        self.cnt += 1

    def addAll(self, inc: int) -> None:
        self.st.update(self.st.root, 0, self.cnt - 1, inc, 0)

    def multAll(self, m: int) -> None:
        self.st.update(self.st.root, 0, self.cnt - 1, m, 1)

    def getIndex(self, idx: int) -> int:
        if idx > self.cnt - 1:
            return -1
        return self.st.query(self.st.root, idx, idx)


# Your Fancy object will be instantiated and called as such:
# obj = Fancy()
# obj.append(val)
# obj.addAll(inc)
# obj.multAll(m)
# param_4 = obj.getIndex(idx)